# CHAPTER 03 : 평가


- 머신러닝의 프로세스 : 데이터 가공/변환 -> 모델 학습/예측 -> **평가(Evaluation)**

> **성능 평가 지표**
>
> 회귀 모델 : 실제값과 예측값의 오차 평균값에 기반 (ex : 오차에 절댓값을 씌운 뒤 평균 오차를 구하거나 오차의 제곱 값에 루트를 씌운 뒤 평균 오차를 구하는 방법 등 기본적으로 예측 오차를 가지고 정규화 수준을 재가공하는 방법)
>
> 분류 모델 : 기본적으로는 실제값과 예측값의 오류가 적은지의 정확도에 기반하지만, 그밖에도 다양한 지표 고려 필요
> - 정확도(Accuracy)
> - 오차행렬(Confusion Matrix)
> - 정밀도(Precision)
> - 재현율(Recall)
> - F1 스코어
> - ROC AUC

## 01 정확도 (Accuracy)

$정확도(Accuracy) = \cfrac{예측결과가 동일한 데이터 건수}{전체 예측 데이터 건수}$

**정확도 지표의 문제점 확인하기**
- 사이킷런의 BaseEstimator 클래스를 상속받아 아무런 학습을 하지 않고, 성별에 따라 생존자를 예측하는 Classifier 생성하기
- (BaseEstimator : Customized 형태의 Estimator를 개발자가 생성할 수 있다.)
- fit() 메서드는 아무것도 수행하지 않으며, 예측을 수행하는 predict()는 단순히 Sex 피처가 1이면 0, 그렇지 않으면 0으로 예측

In [2]:
from sklearn.base import BaseEstimator
import numpy as np

class MyDummyClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range (X.shape[0]) :
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else : 
                pred[i] = 1
        return pred

In [3]:
## 타이타닉 예측을 위한 전처리 함수 정의
# Null 처리 함수
def fillna(df) :
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행
from sklearn.preprocessing import LabelEncoder
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

#데이터 전처리 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [4]:
# MyDummyClassifier를 이용해 타이타닉 생존자 예측 수행
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터/테스트 데이터 분할
titanic_df = pd.read_csv('./data/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test=train_test_split(X_titanic_df, y_titanic_df,
                                                 test_size = 0.2, random_state = 0)

# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행
myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)
mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는 : {0:.4f}'.format(accuracy_score(y_test, mypredictions)))
print('해석 : 단순한 알고리즘으로 예측해도 정확도는 높을 수 있다. 따라서 정확도만을 평가 지표로 사용하는 것은 신중해야 한다.')

Dummy Classifier의 정확도는 : 0.7877
해석 : 단순한 알고리즘으로 예측해도 정확도는 높을 수 있다. 따라서 정확도만을 평가 지표로 사용하는 것은 신중해야 한다.


**MNIST 데이터셋을 활용해 정확도 지표 적용의 문제점 확인하기**
- MNIST 데이터셋은 원래 0부터 9까지의 숫자 이미지 픽셀 정보를 기반으로, 레이블 값이 0부터 9까지 있는 멀티 레이블 분류를 위한 것이나, 여기서는 레이블 값이 7인 것만 True, 나머지는 False로 변환해 이진 분류 문제로 변형해 확인하도록 한다.

In [5]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    
    # 입력값으로 들어오는 X 데이터셋의 크기만큼 모두 0값으로 만들어 변환
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

# 사이킷런 내장 데이터셋인 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환, 7이 아니면 False이고 0으로 변환
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

In [6]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 데이터 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
print('모든 예측을 단순히 0으로 했을 때 정확도는 : {:.3f}'.format(accuracy_score(y_test, fakepred)))

레이블 데이터 세트 크기 : (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 단순히 0으로 했을 때 정확도는 : 0.900


## 02 오차행렬 (Confusion Matrix)

- 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고(confused) 있는지 보여주는 지표
- 이진 분류의 예측 오류에 대해 파악할 수 있는 지표

> TN : 예측값을 Negative 값 0으로 예측, 실제 값 역시 Negative 값 0
>
> FP : 예측값을 Positive 값 1로 예측, 실제 값은 Negative 값 0
>
> FN : 예측값을 Negative 값 0으로 예측, 실제 값 역시 Positive 값 1
>
> TP : 예측값을 Positive 값 0으로 예측, 실제 값 역시 Positive 값 1

In [7]:
# 앞선 예제의 MyFakeClassifer의 예측 성능 지표를 오차 행렬로 표현하기
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_test, fakepred)
print('confusion_matrix (array) :\n', conf_mat)
print()
print('TN : {},  FP : {} \nFN : {},  TP : {}'.format(conf_mat[0,0], conf_mat[0,1], conf_mat[1,0], conf_mat[1,1]))

confusion_matrix (array) :
 [[405   0]
 [ 45   0]]

TN : 405,  FP : 0 
FN : 45,  TP : 0


> 정확도 (Accuracy) : (TN + TP) / (TN + FP + FN + TP)  --- 전체 데이터 수 중 예측 값이 실제 값과 동일한 건수 비율
> 
> 정밀도 (Precision) : TP / (FP + TP)  --- 예측을 Positive로 한 대상 중 실제 값도 Positive로 일치한 데이터 비율
> 
> 재현율 (Recall) : TP / (FN + TP)  --- 실제 값이 Positive인 대상 중에 예측 값도 Positive로 일치한 데이터 비율 (=민감도 Sensitivity 또는 TPR)

- 재현율이 중요 지표인 경우 : 실제 Positive 양성 데이터를 Negative로 잘못 판단하면 업무상 큰 영향이 발생하는 경우 (ex : 암 판단, 보험 사기 적발 모델)
- 정밀도가 중요 지표인 경우 : 실제 Negative인 데이터를 Positive로 잘못 판단하면 업무상 큰 영향이 발생하는 경우 (ex : 스팸 메일 분류)]
- **두 지표 모두 TP를 높이는 데 초점을 맞추지만, 재현율은 FN을, 정밀도는 FP를 낮추는 데 초점을 맞춘다는 차이**

#### 오차 행렬 및 정밀도, 재현율을 모두 구해서 예측 성능 평가하기
- 정밀도 계산 : precision_score() API 사용
- 재현율 계산 : recall_score() API 사용
- get_clf_eval() : confusion matrix, accuracy, precision, recall 등의 평가를 한 번에 호출하는 함수 정의해 실습

In [8]:
# 위 예제에 이어서 작성
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}'.format(accuracy, precision, recall))

In [9]:
# 로지스틱 회귀 기반으로 타이타닉 생존자 예측 후, 평가 지표 호출
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# 원본 데이터를 재로딩, 데이터 가공, 학습 데이터와 테스트 데이터로 분할
titanic_df = pd.read_csv('./data/titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                   test_size = 0.20, random_state = 11)
lr_clf = LogisticRegression()

lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확도 : 0.8492, 정밀도 : 0.7742, 재현율 : 0.7869


### 정밀도/재현율 트레이드오프
- 분류의 결정 임곗값(Threshold)을 조정해 정밀도 또는 재현율의 수치를 높일 수 있다.
- 이 때, 정밀도와 재현율은 상호 보완적 지표이기 때문에 어느 한쪽을 높이면 다른 수치는 떨어지기 쉬운데, 이를 Trade-off라고 한다.

- predict_proba() : 학습이 완료된 사이킷런 Classifier 객체에서 호출이 가능하며, 테스트 피처 데이터 세트를 파라미터로 입력 시 테스트 피처 레코드의 개별 클래스 예측 확률을 반환

In [11]:
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)
print('pred_proba() 결과 Shape : {0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 :\n', pred_proba[:3])

# 예측 확률 array와 예측 결괏값 array를 병합(concatenate)해 예측 확률과 결괏값 한눈에 확인
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1, 1)], axis=1)
print('두 개의 class 중 더 큰 확률을 클래스 값으로 예측 \n', pred_proba_result[:3])

pred_proba() 결과 Shape : (179, 2)
pred_proba array에서 앞 3개만 샘플로 추출 :
 [[0.4623509  0.5376491 ]
 [0.87875882 0.12124118]
 [0.87717457 0.12282543]]
두 개의 class 중 더 큰 확률을 클래스 값으로 예측 
 [[0.4623509  0.5376491  1.        ]
 [0.87875882 0.12124118 0.        ]
 [0.87717457 0.12282543 0.        ]]
